# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [7]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

customer_products.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [9]:
pivot_table = customer_products.pivot_table(values='Quantity', columns='CustomerID', index='ProductName', aggfunc='sum').fillna(0)
pivot_table.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [12]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot_table.T, 'euclidean'))), index=pivot_table.columns, columns=pivot_table.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [13]:
custID = 412
similar_cust = list(distances[custID].sort_values(ascending=False)[1:].head().index)
similar_cust

[3317, 3535, 1008, 1072, 639]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [16]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
sim_cust_prod.head()

,CustomerID,ProductName,Quantity
452,639,Anchovy Paste - 56 G Tube,1
453,639,Appetizer - Mushroom Tart,1
454,639,Assorted Desserts,1
455,639,Bread - Italian Roll With Herbs,1
456,639,Bread - Raisin Walnut Oval,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [18]:
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products

,ProductName,Quantity
0,Olive - Spread Tapenade,3
1,Sprouts - Baby Pea Tendrils,3
2,Wine - Blue Nun Qualitatswein,3
3,"Veal - Inside, Choice",3
4,"Pepper - Black, Whole",3
5,Soup - Campbells Bean Medley,3
6,"Sauce - Gravy, Au Jus, Mix",3
7,Cheese - Cambozola,3
8,Soupfoamcont12oz 112con,2
9,Squid - Tubes / Tenticles 10/20,2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [20]:
merged = pd.merge(ranked_products, pd.DataFrame(pivot_table[custID]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recommendations = merged[merged['Purchased']==0].head()
recommendations

,ProductName,Quantity,Purchased
0,Olive - Spread Tapenade,3,0.0
1,Sprouts - Baby Pea Tendrils,3,0.0
2,Wine - Blue Nun Qualitatswein,3,0.0
4,"Pepper - Black, Whole",3,0.0
5,Soup - Campbells Bean Medley,3,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [25]:
recommendations = {}
customers = list(data['CustomerID'].unique())

for customer in customers:
    similar_cust = list(distances[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(pivot_table[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [26]:
rec_df = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
rec_df.columns = ['CustomerID', 'Reco1', 'Reco2', 'Reco3', 'Reco4', 'Reco5']
rec_df

,CustomerID,Reco1,Reco2,Reco3,Reco4,Reco5
0,61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
1,77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
3,23548,Squid - Tubes / Tenticles 10/20,Sprouts - Baby Pea Tendrils,Flavouring - Orange,Bread - Raisin Walnut Oval,Banana Turning
4,78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
5,83106,Cocoa Butter,"Cheese - Boursin, Garlic / Herbs",Garlic - Peeled,Cheese - Mix,Sauce - Hollandaise
6,11253,Juice - Lime,Tomatoes Tear Drop,Pomello,"Potatoes - Instant, Mashed",Table Cloth - 53x69 Colour
7,35107,Bread - French Baquette,Bandage - Flexible Neon,"Lentils - Red, Dry",Thermometer Digital,Wine - Hardys Bankside Shiraz
8,15088,Soup Knorr Chili With Beans,Soupfoamcont12oz 112con,Veal - Sweetbread,Cocktail Napkin Blue,Bread - Raisin Walnut Oval
9,26031,Cheese - Mozzarella,Bananas,Quiche Assorted,"Lemonade - Natural, 591 Ml",Coffee - Hazelnut Cream


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [31]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot_table.T, 'cosine'))), index=pivot_table.columns, columns=pivot_table.columns)
#distances

In [32]:
recommendations = {}
customers = list(data['CustomerID'].unique())

for customer in customers:
    similar_cust = list(distances[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(pivot_table[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])
    
rec_df = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
rec_df.columns = ['CustomerID', 'Reco1', 'Reco2', 'Reco3', 'Reco4', 'Reco5']
rec_df

,CustomerID,Reco1,Reco2,Reco3,Reco4,Reco5
0,61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,77352,Bread - Rye,Fondant - Icing,Lettuce - Frisee,Bread - French Baquette,Pork - Kidney
2,40094,Juice - V8 Splash,Beef - Inside Round,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Cake - Mini Cheesecake
3,23548,Juice - Orange,Bagel - Plain,Olives - Stuffed,Veal - Sweetbread,Cinnamon Buns Sticky
4,78981,Wine - Blue Nun Qualitatswein,Cookies - Assorted,Cheese - Cottage Cheese,Baking Powder,Quiche Assorted
5,83106,Bouq All Italian - Primerba,Onions - Vidalia,Scallops - Live In Shell,Pastry - Choclate Baked,"Soup - Canadian Pea, Dry Mix"
6,11253,"Nut - Chestnuts, Whole","Beef - Tenderlion, Center Cut",Veal - Eye Of Round,"Soup - Canadian Pea, Dry Mix",Wine - Gato Negro Cabernet
7,35107,Cheese Cloth No 100,"Wine - Cahors Ac 2000, Clos",Butter - Unsalted,"Bread - Roll, Canadian Dinner",Onions - Cippolini
8,15088,Veal - Sweetbread,Ecolab - Lime - A - Way 4/4 L,Island Oasis - Mango Daiquiri,Vaccum Bag 10x13,Milk - 2%
9,26031,Foam Cup 6 Oz,Spinach - Baby,Cookies - Assorted,Sardines,Cheese - Parmesan Grated


In [ ]:
# The product recommandations are not the same anymore with the "Cosine" method, compared to the "Euclidian" method.